#Import

In [ ]:
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pickle
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import random
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [ ]:
train = total_data[['날짜','종목코드','종가', '시가_결측치있음','고가_결측치있음','저가_결측치있음','거래량_결측치있음',
                    'VWAP_결측치있음', 'VWAP_차이_결측치있음', 'usdtokrw']] # 사용할 피처 선택
train['날짜'] = pd.to_datetime(train['날짜'])
train = train.fillna(0) #결측치 처리 > 시가 고가 저가의 결측치의 경우 거래정지로 인한 값이기 때문에 0으로 처리
train.isna().sum()

날짜               0
종목코드             0
종가               0
시가_결측치있음         0
고가_결측치있음         0
저가_결측치있음         0
거래량_결측치있음        0
VWAP_결측치있음       0
VWAP_차이_결측치있음    0
usdtokrw         0
dtype: int64

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Reshape
from keras.layers import Dropout

#CNN 모델
def cnn_model():
    model = Sequential()
    model.add(Reshape((9, 1), input_shape=(9,))) #위에 선정한 피처 10개 중, target인 종가를 제외한 9개
    model.add(Conv1D(32, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Dropout(0.1))

    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Dropout(0.1))

    model.add(Conv1D(128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.1))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    return model

In [ ]:
# 반복적으로 학습 및 예측 수행
train_start_date = train['날짜'].min()
train_end_date = train_start_date + pd.DateOffset(days=109)
test_start_date = train_end_date + pd.DateOffset(days=1)
test_end_date = test_start_date + pd.DateOffset(days=21)
# 예측일을 이렇게 나눈 이유 : 7.28일까지의 데이터는 2021-06-01 ~ 2023-07-28 > 787일이다.
# 위와같이 DateOffset을 통해 구분을 했기 때문에 학습기간을 타겟시킬수가 없어서 그냥 786일을 소인수분해하여 6*131 한 값을 사용
# 131일 중, 테스트 기간을 제외한 기간을 train에 입력 (131-22 = 109)

results_df = pd.DataFrame(columns=['종목코드', '절대값', '차이'])

while test_end_date <= pd.to_datetime('2023-07-28'):  # 최종 예측일까지 반복
    # 학습 데이터 설정
    train_data = train[(train['날짜'] >= train_start_date) & (train['날짜'] <= train_end_date)]
    train_features = train_data.drop(['종가'], axis=1)
    labels = train_data['종가']

    # 날짜 데이터를 숫자형으로 변환
    train_features['날짜'] = train_features['날짜'].dt.strftime('%Y%m%d').astype(np.int32)

    # 스케일링
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(train_features)

    # 모델 학습
    model = cnn_model()
    model.fit(features_scaled, labels)

    # 예측 데이터 설정
    test_data = train[(train['날짜'] >= test_start_date) & (train['날짜'] <= test_end_date)]
    test_features = test_data.drop(['종가'], axis=1)

    # 날짜 데이터를 숫자형으로 변환
    test_features['날짜'] = test_features['날짜'].dt.strftime('%Y%m%d').astype(np.int32)

    # 스케일링
    test_features_scaled = scaler.transform(test_features)

    # 예측
    predictions = model.predict(test_features_scaled)

    # 종목 코드별로 결과 저장
    for i, pred in enumerate(predictions):
        actual_close = test_data.iloc[i]["종가"]
        print(f'일자: {test_data.iloc[i]["날짜"]}, 종목 코드: {test_data.iloc[i]["종목코드"]}, 예측 종가: {pred}, 실제 종가: {actual_close}')
        if actual_close != 0:
            return_rate = (pred - actual_close) / actual_close

            # 결과를 DataFrame에 추가 (종목 코드별로 하나씩)
            if test_data.iloc[i]["종목코드"] not in results_df['종목코드'].values:
                results_df = results_df.append({'종목코드': test_data.iloc[i]["종목코드"], '절대값': abs(return_rate), '차이': return_rate}, ignore_index=True)
            else:
                results_df.loc[results_df['종목코드'] == test_data.iloc[i]["종목코드"], '절대값'] += abs(return_rate)
                results_df.loc[results_df['종목코드'] == test_data.iloc[i]["종목코드"], '차이'] += return_rate

    # 다음 구간 설정
    train_start_date += pd.DateOffset(days=131)
    train_end_date += pd.DateOffset(days=131)
    test_start_date += pd.DateOffset(days=131)
    test_end_date += pd.DateOffset(days=131)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
일자: 2023-07-18 00:00:00, 종목 코드: 227840, 예측 종가: [11625.271], 실제 종가: 11800
일자: 2023-07-19 00:00:00, 종목 코드: 227840, 예측 종가: [11458.14], 실제 종가: 11600
일자: 2023-07-20 00:00:00, 종목 코드: 227840, 예측 종가: [11414.475], 실제 종가: 11610
일자: 2023-07-21 00:00:00, 종목 코드: 227840, 예측 종가: [11385.26], 실제 종가: 11640
일자: 2023-07-24 00:00:00, 종목 코드: 227840, 예측 종가: [11337.243], 실제 종가: 11330
일자: 2023-07-25 00:00:00, 종목 코드: 227840, 예측 종가: [11218.497], 실제 종가: 11110
일자: 2023-07-26 00:00:00, 종목 코드: 227840, 예측 종가: [11033.563], 실제 종가: 10790
일자: 2023-07-27 00:00:00, 종목 코드: 227840, 예측 종가: [10938.61], 실제 종가: 11010
일자: 2023-07-06 00:00:00, 종목 코드: 227950, 예측 종가: [5082.7925], 실제 종가: 1079
일자: 2023-07-07 00:00:00, 종목 코드: 227950, 예측 종가: [5044.1147], 실제 종가: 1068
일자: 2023-07-10 00:00:00, 종목 코드: 227950, 예측 종가: [5066.0796], 실제 종가: 1067
일자: 2023-07-11 00:00:00, 종목 코드: 227950, 예측 종가: [5079.73], 실제 종가: 1079
일자: 2023-07-12 00:00:00, 종목 코드: 227950, 예측 종가: [5061.2397], 실제 종가: 1063
일자: 2023-07-13 00:00:00, 

#순위 선정 방식

오차의 절대값이 작을수록 잘 예측했다는 것이고
잘 예측한다는 것은 그만큼 변동성이 작고 가격이 신뢰할만하다고 판단하여 매수하고,
반대로 변동성이 크고 가격이 신뢰하기 힘들다고 보이는 종목들은 과대평가 되었다고 생각하여 공매도 하기로 하였습니다.

In [ ]:
# 결과값에서 절대값이 의미있다고 판단
# 절대값이 작을 수록 모델이 비교적 정확하게 맞췄다는 것을 의미함.
# 절대값이 작은 종목 기준으로 순위를 선정.
results_df = results_df.sort_values(by='절대값', ascending=True)
results_df['종목코드'] = 'A' + results_df['종목코드']
results_df['순위'] = results_df['절대값'].rank(method='first', ascending=True).astype('int') # 각 순위를 중복없이 생성
results_df

,종목코드,절대값,차이,순위
1345,A107590,[0.9754218773011065],[0.09581984181814246],1
145,A003240,[1.057153976874107],[-0.563745975177767],2
694,A035760,[1.1440965089516815],[-0.1599568618068966],3
681,A034950,[1.184169456950296],[-0.31104013278095866],4
32,A000670,[1.2074919485279818],[-0.7053753863377185],5
...,...,...,...,...
190,A004410,[944.8352],[944.8352],1996
1223,A091090,[1260.4727],[1260.4727],1997
1322,A102280,[1527.752],[1527.752],1998
1506,A159910,[3087.2036],[3087.2036],1999


In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')
sample_submission

,종목코드,순위
0,A000020,1
1,A000040,2
2,A000050,3
3,A000070,4
4,A000080,5
...,...,...
1995,A375500,1996
1996,A378850,1997
1997,A383220,1998
1998,A383310,1999


In [ ]:
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')
baseline_submission

,종목코드,순위
0,A000020,879
1,A000040,1931
2,A000050,594
3,A000070,76
4,A000080,287
...,...,...
1995,A375500,224
1996,A378850,1793
1997,A383220,213
1998,A383310,222


In [ ]:
baseline_submission.to_csv('cnn_submission_result_last.csv', index=False)